In [156]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 256 kB in 4s (59.6 kB/s)
Reading 

In [157]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-15 18:35:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.5’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.67MB/s    in 0.2s    

2022-05-15 18:35:35 (5.67 MB/s) - ‘postgresql-42.2.9.jar.5’ saved [914037/914037]



In [158]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [159]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [160]:
user_data_df.printSchema()
user_data_df.count()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



341931

In [161]:
#Total number of records
user_data_df.count()

341931

In [162]:
user_data_df.createOrReplaceTempView("user_data_view")

In [163]:
#Unique values


user_data_summary_df = spark.sql('''
SELECT
    product_title, AVG(star_rating) as star_rating_avg, AVG(helpful_votes) as helpful_votes_avg, AVG(total_votes) as total_votes_avg
FROM
    user_data_view GROUP BY product_title
ORDER BY
    star_rating_avg DESC ''')
user_data_summary_df.show()


+--------------------+---------------+------------------+------------------+
|       product_title|star_rating_avg| helpful_votes_avg|   total_votes_avg|
+--------------------+---------------+------------------+------------------+
|Atari Deer Hunter...|            5.0|               0.0|               1.0|
|Cakewalk SONAR X3...|            5.0|               4.0|               4.0|
|101 Photoshop Les...|            5.0|               3.0|               3.0|
|    TeachMe! Finnish|            5.0|               0.0|               0.0|
|Symantec Backup E...|            5.0|               1.0|               1.0|
|Resume Maker Prof...|            5.0|               1.0|               1.0|
|REAL-Mentor: Real...|            5.0|               1.0|               1.0|
|OMNIPAGE 18.0 US ...|            5.0|               2.0|               3.0|
|Fluenz Mandarin 1...|            5.0|               4.0|               5.0|
|Kaspersky Antivir...|            5.0|               0.0|               0.0|

In [164]:
user_data_df.groupBy().avg().show()

+--------------------+-------------------+------------------+------------------+-----------------+
|    avg(customer_id)|avg(product_parent)|  avg(star_rating)|avg(helpful_votes)| avg(total_votes)|
+--------------------+-------------------+------------------+------------------+-----------------+
|3.2607869498945694E7|5.006618467260734E8|3.5670354545215264| 4.362482489157169|5.503203862767634|
+--------------------+-------------------+------------------+------------------+-----------------+



In [165]:
user_data_df.groupBy().avg().show()

+--------------------+-------------------+------------------+------------------+-----------------+
|    avg(customer_id)|avg(product_parent)|  avg(star_rating)|avg(helpful_votes)| avg(total_votes)|
+--------------------+-------------------+------------------+------------------+-----------------+
|3.2607869498945694E7|5.006618467260734E8|3.5670354545215264| 4.362482489157169|5.503203862767634|
+--------------------+-------------------+------------------+------------------+-----------------+



# Drop if the total votes =0 or purchase in not verified

In [166]:
user_data_fake_df = spark.sql('''
SELECT 
*
FROM user_data_view where  (verified_purchase == "N") 
''')

In [167]:
user_data_fake_df.createOrReplaceTempView("user_data_fake_view")


In [168]:
user_data_fake_summary_df = spark.sql('''
SELECT
    product_title, AVG(star_rating) as star_rating_avg, AVG(helpful_votes) as helpful_votes_avg, AVG(total_votes) as total_votes_avg
FROM
    user_data_fake_view GROUP BY product_title
ORDER BY
    star_rating_avg DESC ''')
user_data_fake_summary_df.show()

+--------------------+---------------+------------------+------------------+
|       product_title|star_rating_avg| helpful_votes_avg|   total_votes_avg|
+--------------------+---------------+------------------+------------------+
|101 Photoshop Les...|            5.0|               3.0|               3.0|
|         Becker Draw|            5.0|               1.0|               1.0|
|    TeachMe! Finnish|            5.0|               0.0|               0.0|
|Linux Jazz 6-disk...|            5.0|               0.5|               0.5|
| Adventure Game Pack|            5.0|               4.0|               4.0|
|Edmark Sammy's Sc...|            5.0|               0.0|               0.0|
|Sun Solaris Admin...|            5.0|               0.0|               0.0|
|Blockbase Pieced ...|            5.0|               1.0|               1.0|
|Lotus Domino R5 D...|            5.0|               0.0|               0.0|
|Chicken Invaders ...|            5.0|               2.0|               2.0|

In [169]:
user_data_fake_df.groupBy().avg().show()

+-------------------+--------------------+------------------+------------------+-----------------+
|   avg(customer_id)| avg(product_parent)|  avg(star_rating)|avg(helpful_votes)| avg(total_votes)|
+-------------------+--------------------+------------------+------------------+-----------------+
|3.616639896305133E7|4.9938716322380435E8|3.2029613628284705| 7.509878045445845|9.293825708895026|
+-------------------+--------------------+------------------+------------------+-----------------+



In [170]:
#Clean up
user_data_df = spark.sql('''
SELECT 
*
FROM user_data_view where  (verified_purchase == "Y") and total_votes>0
''')
#Drop duplicates
user_data_df=user_data_df.dropDuplicates()
user_data_df.count()

95683

In [171]:
user_data_df.createOrReplaceTempView("user_data_cleaned_view")

In [172]:
user_data_summary1_df = spark.sql('''
SELECT
    product_title, AVG(star_rating) as star_rating_avg, AVG(helpful_votes) as helpful_votes_avg, AVG(total_votes) as total_votes_avg
FROM
    user_data_cleaned_view GROUP BY product_title
ORDER BY
    star_rating_avg DESC ''')
user_data_summary1_df.show()

+--------------------+---------------+-----------------+---------------+
|       product_title|star_rating_avg|helpful_votes_avg|total_votes_avg|
+--------------------+---------------+-----------------+---------------+
|Award Banner & Ce...|            5.0|              2.0|            3.0|
|   Enemy Infestation|            5.0|              2.0|            2.0|
|Trend Micro Titan...|            5.0|             0.25|            1.5|
|Rosetta Stone Fre...|            5.0|             10.0|           12.0|
|431312 QuickBooks...|            5.0|              1.0|            1.0|
|Resume Maker Prof...|            5.0|              1.0|            1.0|
|    Amazon Trail 3.0|            5.0|              1.0|            1.0|
|ExpectantSee: You...|            5.0|              2.0|            2.0|
|Carnivores - City...|            5.0|              1.5|            2.5|
|Starry Night Back...|            5.0|              1.0|            2.0|
|Dive Cd-ROM for S...|            5.0|             

In [173]:
user_data_df.groupBy().avg().show()

+--------------------+-------------------+-----------------+------------------+-----------------+
|    avg(customer_id)|avg(product_parent)| avg(star_rating)|avg(helpful_votes)| avg(total_votes)|
+--------------------+-------------------+-----------------+------------------+-----------------+
|3.1223754642183043E7|4.915491732530544E8|3.521827283843525| 4.108284648265627|5.457374873279475|
+--------------------+-------------------+-----------------+------------------+-----------------+



In [174]:
#Review id table. Date is in yyyy-mm-dd but prefer to have a way to enforce it
# import org.apache.spark.sql.functions.{to_date, to_timestamp}
review_id_table_df = user_data_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
# review_id_table_df = review_id_table_df.withColumn("review_date", to_date($"review_date", "yyyy-mm-dd"))
review_id_table_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R11FLCMCYFGDHO|   28160256|B001ALLVMO|     674500841| 2015-08-17|
|R220DIOG4JR3JL|   27460799|B009SPN118|     244120695| 2015-08-12|
|R1D0STJ204W0LW|   53026486|B00OW2P7HS|     705811299| 2015-08-11|
|R1T3QWHTHGO6JG|   47240862|B007TMFD7M|     101191858| 2015-08-10|
|R2WXS6LL0A1I5R|    1675649|B00EXPSW2Q|     880437537| 2015-08-10|
| R3W7O313FHNBH|   24417466|B00LQZ95RC|     299353569| 2015-08-10|
| R66MM0LMAFQ78|   10055799|B00OBQ6PIK|     190919986| 2015-07-22|
|R2BIWFVIRALDDO|   21157171|B00N4OLCRO|     776572654| 2015-07-22|
|R1PFXHNCRR6G3S|   37631715|B00R9NXYGI|     594480361| 2015-07-19|
|R13DBDWVT0VIFO|   11180940|B00MV9EL6M|     482986152| 2015-07-17|
|R3TZPCDY7B8AAW|   29446916|B00MV9EKPO|     403400265| 2015-07-09|
|R2WT3VW9LI02ZT|     771851|B00LOUGJO6|     801357787| 2015-07

In [175]:
#vine table. No changes needed
vine_table_df=user_data_df.select(["review_id", "star_rating","helpful_votes", "total_votes","vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R11FLCMCYFGDHO|          5|            1|          1|   N|
|R220DIOG4JR3JL|          3|            1|          1|   N|
|R1D0STJ204W0LW|          5|            0|          1|   N|
|R1T3QWHTHGO6JG|          2|            1|          3|   N|
|R2WXS6LL0A1I5R|          1|            1|          1|   N|
| R3W7O313FHNBH|          1|           13|         13|   N|
| R66MM0LMAFQ78|          1|            1|          2|   N|
|R2BIWFVIRALDDO|          1|            2|          3|   N|
|R1PFXHNCRR6G3S|          5|            1|          1|   N|
|R13DBDWVT0VIFO|          5|            1|          1|   N|
|R3TZPCDY7B8AAW|          1|            2|          2|   N|
|R2WT3VW9LI02ZT|          2|            2|          2|   N|
|R2EJLG6IA3MVNA|          1|            0|          1|   N|
|R2EYUHNIS76T1A|          5|            

In [176]:
#Customer id and their  count
df=user_data_df.select(["customer_id"])
customers_df=df.groupBy("customer_id").count()
customers_df=customers_df.withColumnRenamed("count","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   30808147|             3|
|   13706099|             2|
|   12167120|             1|
|   50769070|             1|
|   33185561|             1|
|   15321104|             1|
|   39341740|             1|
|   20436576|             1|
|   15324233|             2|
|   42966770|             1|
|    4032288|             1|
|   16982731|             1|
|   36998253|             1|
|   49107143|             1|
|   26407568|             2|
|   52869911|             1|
|   51938979|             1|
|   21448041|             1|
|   14730274|             1|
|   11938892|             1|
+-----------+--------------+
only showing top 20 rows



In [177]:
#Verify the schema matches with what was provided. 

mode = "overwrite"  #Cant be override
jdbc_url="jdbc:postgresql://database-1.cbrsr6wychlr.us-west-1.rds.amazonaws.com:5432/software_db"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [178]:
review_id_table_df.createOrReplaceTempView("review_id_view1")
review_id_table_df = spark.sql('''
SELECT 
review_id, customer_id, product_id, product_parent, DATE(review_date)
FROM review_id_view1
''')
review_id_table_df1=review_id_table_df.select('review_id').distinct()

review_id_table_df1.printSchema()

root
 |-- review_id: string (nullable = true)

